# Create READII negative control CTs to run through FMCIB

This code utilizes the RADCURE dataset from TCIA. This dataset is under the TCIA Restricted License, so users will need to request access prior to running this code.

We will be using the RADCURE test subset specified in the clinical data sheet. 

## Set up pixi environment kernel

1. Run the following commands in the terminal:

    ```bash
    $ pixi install -e default

    $ pixi run kernel
    ```

2. In the `Select Kernel` menu at the top right of the notebook, select `Jupyter Kernel` as the source. 

3. Refresh the options and one called `readii-fmcib` should appear. Select this option.

## Imports

In [1]:
import itertools
import yaml

from imgtools.autopipeline import AutoPipeline
from pathlib import Path
from readii.io.loaders import loadImageDatasetConfig

import sys; sys.path.append("code")
from process_crop import prep_data_for_fmcib

## Settings for processing

In [11]:
config = loadImageDatasetConfig("RADCURE", Path("config"))

DATASET_NAME = config["dataset_name"]

ROI_PATTERN = config["roi_pattern"]
ROI_NAME = list(ROI_PATTERN.keys())[0]

MODALITIES = config['modalities']

NEG_CONTROL_REGIONS = config["negative_control_regions"]
NEG_CONTROL_TYPES = config["negative_control_types"]

## Set up data directories

In [3]:
for combo in itertools.product(["rawdata", "procdata"], [DATASET_NAME], ["clinical", "images"]):
    Path(*combo).mkdir(parents=True, exist_ok=True)

raw_images_dir = Path("rawdata", DATASET_NAME , "images")
proc_images_dir = Path("procdata", DATASET_NAME, "images")

# Directory for output of MIT run
nifti_dir = proc_images_dir / "converted_niftis"

## Make ROI yaml file to use with Med-ImageTools

In [4]:
roi_yaml_path = raw_images_dir / "mit_roi_names.yaml"

if not roi_yaml_path.exists():
    with open(roi_yaml_path, "w") as outfile:
        yaml.dump(ROI_PATTERN, outfile)

## Run Med-ImageTools on downloaded data to index and convert to niftis

**Put path to directory containing the image data below:**

In [5]:
# Path to the directory containing the patient ID level directories of the images
path_to_downloaded_data = Path("INSERT_PATH_HERE")

In [ ]:
pipeline = AutoPipeline(input_directory=path_to_downloaded_data,
                        output_directory=nifti_dir,
                        modalities=MODALITIES,
                        spacing=(0.,0.,0.),
                        read_yaml_label_names = True,
                        ignore_missing_regex = True,
                        roi_yaml_path = roi_yaml_path
                        )

pipeline.run()

# FMCIB Input Prep

### READII Negative Controls

Create each type of negative control specified by `NEGATIVE_CONTROL_REGIONS` and `NEGATIVE_CONTROL_TYPES` using `READII`

### Crop and resize for FMCIB expected input size

Crop and resize the images to `FMCIB_INPUT_SIZE` (can be set in next cell) with one of three `CROP_METHOD` options:

1. `bbox` - Find bounding box based on dimensions of the region of interest (ROI), crop image to these coordinates, resize/resample to `FMCIB_INPUT_SIZE`.
2. `cube` - Create cube based on largest ROI bounding box dimension, crop image to these coordinates, resize/resample to `FMCIB_INPUT_SIZE`.
3. `centroid` - Create `FMCIB_INPUT_SIZE` cube centered on the ROI centroid, crop to these coordinates.

#### *All of these settings can be set in the `config/DATASET_NAME.yaml` file.*

In [13]:
CROP_METHOD = config['crop_method'] # Options are bbox, cube, centroid
FMCIB_INPUT_SIZE = tuple(config['crop_size'])

This will only run on one crop method at a time. To run on each crop method, the `config` file must be updated and reloaded.

In [ ]:
# Crop and resize the original images
print(f"Processing original images...")
original_image_df = prep_data_for_fmcib(input_image_dir = nifti_dir,
                                        output_dir_path = proc_images_dir,
                                        crop_method = CROP_METHOD,
                                        input_size = FMCIB_INPUT_SIZE,
                                        roi_name = ROI_NAME,
                                        negative_control_strategy = "original",
                                        parallel = True
                                        )

# Make negative control, then crop and resize images
for type in NEG_CONTROL_TYPES:
    for region in NEG_CONTROL_REGIONS:
        print(f"Creating and processing {type} {region} negative control...")
        if region == "non_roi":
            # Non_roi processing takes us too much memory and fails in parallelization
            parallel = False
        else: 
            parallel = True
        
        neg_control_df = prep_data_for_fmcib(input_image_dir = nifti_dir,
                                             output_dir_path = proc_images_dir,
                                             crop_method = CROP_METHOD,
                                             input_size = FMCIB_INPUT_SIZE,
                                             roi_name = ROI_NAME,
                                             negative_control_strategy = type,
                                             negative_control_region = region,
                                             parallel = parallel)